## AraBert / AraGPT Model
* Github model source link : https://github.com/aub-mind/arabert

# AraELECTRA Model
* use the detector
* si un mot n'est pas similaire au mots générés par le modele ,alors il peut contenir une erreur

In [5]:
from transformers import pipeline
from arabert.preprocess import ArabertPreprocessor

#MODEL_NAME = 'aubmindlab/aragpt2-base' error
#MODEL_NAME = 'aubmindlab/Araelectra-base-discriminator ' error
#MODEL_NAME = 'aubmindlab/Araelectra-base-generator'
MODEL_NAME = 'aubmindlab/bert-base-arabertv02-twitter'


In [6]:
MODEL_NAME = 'aubmindlab/Araelectra-base-generator'
fill_mask = pipeline(task="fill-mask",model=MODEL_NAME,tokenizer=MODEL_NAME)
#print(fill_mask(" عاصمة لبنان هي [MASK] " ))
#result : 
#[
#{'score': 0.18456363677978516, 'token': 2079, 'token_str': 'بيروت', 'sequence': 'عاصمة لبنان هي بيروت'},
#{'score': 0.10975801199674606, 'token': 31, 'token_str': ':', 'sequence': 'عاصمة لبنان هي :'},
#{'score': 0.039298899471759796, 'token': 911, 'token_str': 'لبنان', 'sequence': 'عاصمة لبنان هي لبنان'},
#{'score': 0.03792791813611984, 'token': 3743, 'token_str': 'باريس', 'sequence': 'عاصمة لبنان هي باريس'},,
#{'score': 0.03269968181848526, 'token': 3306, 'token_str': 'القدس', 'sequence': 'عاصمة لبنان هي القدس'}
# ]


In [7]:
#-- testing the method
import util as ut
text= "حقق منتخبنا الوطني الاول لكرة [MASK] كل اهدافه المطلوبة"
mask = "القسم"
words = fill_mask("حقق منتخبنا الوطني الاول لكرة [MASK] كل اهدافه المطلوبة")
#words = "حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة"
s=[]
t=[]
for e in words:
    t.append(e['token_str'])
    s.append(e['sequence'])
print("you wrote : ", mask)
print("did you mean : ",ut.lookslike(t,[mask]),"?")


you wrote :  القسم
did you mean :  القدم ?


In [8]:
def replace(mask, text):
    text = text.split()
    text = list(map(lambda x: x.replace(mask, '[MASK]'), text))
    text = ' '.join(str(x) for x in text)
    return(text)


In [41]:
import util as ut
text = "حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة"

def analyse(text):
    ok =True
    res=[]
    print("your text = ",text,"\n")
    for mask in text.split():
        masked_text = replace(mask,text)
        words = fill_mask(masked_text)
        s=[]
        t=[]
        for e in words:
    #        t.append(e['token_str'])
            t.append(ut.clean_str(e['token_str']))
            s.append(e['sequence'])
        correct = ut.lookslike(t, [ut.clean_str(mask)]) #les mots parmi les mots générs qui ressemblent au mot à analyser
    #    correct = ut.lookslike(t, [mask])
    #    if correct != ut.clean_str(mask):
        if correct != mask and correct:
            print("you wrote : [ ", mask , "] did you mean : [ ",correct,"] ?")
            ok = False
            res.append((mask,correct))
    if ok :
        print("This sentence contains 0 errors ")
    return(res)

In [42]:
analyse(text)

your text =  حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة 

you wrote : [  حقق ] did you mean : [  وحقق ] ?
you wrote : [  منتخبنا ] did you mean : [  لمنتخبنا ] ?
you wrote : [  القسم ] did you mean : [  القدم ] ?


[('حقق', 'وحقق'), ('منتخبنا', 'لمنتخبنا'), ('القسم', 'القدم')]

# Evaluation du modele 
*  si y_test n'apprtient pas à la liste générée res => loss

In [43]:
# parcours de la data 
# test 
#evaluation


# combinaison de modeles AraVec et AraELECTRA 
* aravec + aralecetra 
* calcul similarités (avec aravec) entres les mots suggérés par (araelectra)
*   araelectra suggere des couples (mot, erreur)
*   aravec retourne la meilleure correction 
*       similarité(منتخبنا, لمنتخبنا)>similarite(القسم,القدم)--> le mot erroné exact est "القسم"

In [38]:
import gensim
import re  # regularexpression
import spacy
import nltk
import util as ut
from gensim import matutils  # utility fnc for pickling, common scipy operations etc
import numpy as np
import math


In [39]:
#load Aravec model 
nlp = spacy.load("./spacy.aravec.model/")
# Define the preprocessing Class


class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = ut.clean_str(text)
        return self.tokenizer(preprocessed)


In [44]:
def combinedModels(text):
    a = analyse(text)
    sim= [nlp(x).similarity(nlp(y)) for x,y in a]
    error = a[sim.index(min(sim))]
    print("\n the exact error is in word",error[0])

execution

In [45]:
combinedModels(text)


your text =  حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة 

you wrote : [  حقق ] did you mean : [  وحقق ] ?
you wrote : [  منتخبنا ] did you mean : [  لمنتخبنا ] ?
you wrote : [  القسم ] did you mean : [  القدم ] ?

 the exact error is in word القسم
